# Using Tester to test FastKafka

In [ ]:
# | hide

from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field
from typing import List, Optional

In order to speed up development and make testing easier, we have implemented the `Tester` class. The `Tester` instance starts InMemory implementation of Kafka broker i.e. there is no need for starting localhost Kafka service for testing FastKafka apps.
The Tester will redirect consumes and produces decorated functions to the InMemory Kafka broker so that you can quickly test FasKafka apps without the need for a running Kafka broker and all its dependencies.

`Note`: if you are using Jupyter Notebooks you need to allow the usage of nested event loops when working with asyncio. This can be done by adding the following code:

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Test message

To showcase the functionalities of FastKafka and illustrate the concepts discussed, we can use a simple test message called `TestMsg`. Here's the definition of the `TestMsg` class:

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


test_msg = TestMsg(msg="signal")

### InMemory Producer and Consumer apps

Let's create two simple `FastKafka` apps. `consumer_app` consumes from the `preprocessed_data` topic and `producer_app` produces messages to the `preprocessed_data` topic.

In [ ]:
from fastapi import FastAPI

consumer_app = FastKafka()


@consumer_app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    print(msg)


producer_app = FastKafka()


@producer_app.produces()
async def to_preprocessed_data(msg: TestMsg) -> TestMsg:
    return msg

By using `Tester([consumer_app, producer_app])`, this two apps will use InMemory Kakfa broker.
`Note`: it is necessary to define parameter and return types in the producer and consumer methods

In [ ]:
test_msg = TestMsg(msg="signal")
async with Tester([consumer_app, producer_app]) as tester:
    # producer_app produces message to the preprocessed_data topic
    await producer_app.to_preprocessed_data(test_msg)
    # assert that consumer_app consumed from the preprocessed_data topic and it was called with test_msg argument
    await consumer_app.awaited_mocks.on_preprocessed_data.assert_called_with(
        test_msg, timeout=5
    )
print("ok")

23-07-04 00:28:17.992 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-07-04 00:28:17.994 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-07-04 00:28:17.995 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-04 00:28:17.996 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-04 00:28:18.025 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-04 00:28:18.026 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-04 00:28:18.027 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
23-07-04 00:28:18.028 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using

In [ ]:
# | hide
# Same e.g. for VSCode
import asyncio


async def async_tests():
    ...


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())

### Consumer app + Tester (producer) app

It is often necessary to implement only the Kakfa consumer, which then subscribes to an already existing topic. The `Tester` enables simple mirroring, i.e. if we have implemented consumer, the Tester will mock the producer methods (and the other way - if the producer is implemented, `Tester` will mock consumer methods).
In this examle we have `consumer_app` which consumes from the `preprocessed_data` topic.

In [ ]:
consumer_app = FastKafka()


@consumer_app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    print(msg)

Here `consumer_app` has implemented `on_preprocessed_data` and `tester` will have consumer mirrored methods i.e. `to_preprocessed_data` (the same way `producer_app` had `to_preprocessed_data` in the previous example)

In [ ]:
async with Tester(consumer_app) as tester:
    # Send the message to the topic preprocessed_data
    await tester.to_preprocessed_data(test_msg)
    # await tester.mirrors[consumer_app.on_preprocessed_data](test_msg) # the same as previous line

    # Tester mirrors consumer_app consumes() and produces() methods (in this example there is only on_preprocessed_data)
    assert (
        tester.to_preprocessed_data == tester.mirrors[consumer_app.on_preprocessed_data]
    )

    # Assert app consumed the message from the preprocessed_data topic
    await consumer_app.awaited_mocks.on_preprocessed_data.assert_called_with(
        test_msg, timeout=5
    )
print("ok")

23-07-03 23:12:35.574 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-07-03 23:12:35.575 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-07-03 23:12:35.585 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:12:35.587 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-03 23:12:35.588 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
23-07-03 23:12:35.589 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
23-07-03 23:12:35.590 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
23-07-03 23:12:35.591 [INFO] f

### Producer app + Tester (consumer) app

In this example `Tester` will mirror producer method the same way it mirrored the consumer method in the previous example

In [ ]:
producer_app = FastKafka()


@producer_app.produces()
async def to_preprocessed_data(msg: TestMsg) -> TestMsg:
    return msg

In [ ]:
async with Tester(producer_app) as tester:
    # Send the message to the topic preprocessed_data
    await producer_app.to_preprocessed_data(test_msg)

    # Assert app consumed the message from the preprocessed_data topic
    await tester.awaited_mocks.on_preprocessed_data.assert_called_with(
        test_msg, timeout=5
    )
print("ok")

23-07-03 23:12:39.636 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-07-03 23:12:39.637 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-07-03 23:12:39.638 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:12:39.638 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-03 23:12:39.650 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
23-07-03 23:12:39.651 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
23-07-03 23:12:39.652 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
23-07-03 23:12:39.653 [INFO] f

### App (producer & consumer)  + Tester (ptoducer & consumer) app #1

In this example `Tester` will mirror producer and consumer methods.

In [ ]:
app = FastKafka()


@app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> List[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return [prediction]

The `app` has `on_preprocessed_data` and `to_predictions` method defined and `tester` will have their mirrored methods: `to_preprocessed_data` and `on_predictions`.

In [ ]:
async with Tester(app) as tester:
    # inside on_preprocessed_data, to_predictions method is called i.e. app will produce message to predictions topic
    await app.on_preprocessed_data(test_msg)
    # Now we can check if tester consumed from the predictions topic
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

    # produce message to preprocessed_data topic
    await tester.to_preprocessed_data(test_msg)
    # app will consume from the preprocessed_data topic
    await app.awaited_mocks.on_preprocessed_data.assert_called_with(test_msg, timeout=5)
    # while consuming from the preprocessed_data topic (in the line above), app will produce a message to predictions topic
    # and tester will cosume this message
    tester.mocks.on_predictions.assert_called()
print("ok")

23-07-03 23:30:07.840 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-07-03 23:30:07.842 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-07-03 23:30:07.845 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:30:07.846 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-03 23:30:07.876 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:30:07.877 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-03 23:30:07.878 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
23-07-03 23:30:07.879 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using

### App (producer & consumer)  + Tester (ptoducer & consumer) app #2

In this example, our `app` has one consumes and two produces methods.
Every time a company hires new `Employee`, the new employee needs to get email address `on_new_employee`. Also, we want to send a welcome to the new employee `to_welcome_message`

In [ ]:
class Employee(BaseModel):
    name: str
    surname: str
    email: Optional[str]


class EmaiMessage(BaseModel):
    sender: str = "info@gmail.com"
    receiver: str
    subject: str
    message: str


app = FastKafka()


@app.consumes()
async def on_new_employee(msg: Employee):
    employee = await to_employee_email(msg)
    await to_welcome_message(employee)


@app.produces()
async def to_employee_email(employee: Employee) -> Employee:
    # generate new email
    employee.email = employee.name + "." + employee.surname + "@gmail.com"
    return employee


@app.produces()
async def to_welcome_message(employee: Employee) -> EmaiMessage:
    message = f"Dear {employee.name},\nWelcome to the company"
    return EmaiMessage(receiver=employee.email, subject="Welcome", message=message)

In [ ]:
employee = Employee(name="John", surname="Jones")

async with Tester(app) as tester:
    # while consuming from the new_employee topic, app writes to the employee_email and welcome_message topic
    await app.on_new_employee(employee)

    # Now we can check if the methods were called
    await tester.awaited_mocks.on_employee_email.assert_called(timeout=5)
    await tester.awaited_mocks.on_welcome_message.assert_called(timeout=5)

print("ok")

23-07-03 23:58:16.962 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-07-03 23:58:16.968 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-07-03 23:58:16.969 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:58:16.970 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-03 23:58:16.972 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:58:16.973 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-03 23:58:17.025 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-03 23:58:17.026 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProduc

In [ ]:
employee = Employee(name="Mickey", surname="Mouse")
async with Tester(app) as tester:
    # produce the message to new_employee topic
    await tester.to_new_employee(employee)
    # Assert app consumed the message
    await app.awaited_mocks.on_new_employee.assert_called_with(employee, timeout=5)

    # If the the previous assert is true (on_new_employee was called),
    # to_employee_email and to_welcome_message were called inside on_new_employee method

    # Now we can check if this two messages were consumed
    await tester.awaited_mocks.on_employee_email.assert_called(timeout=5)
    await tester.awaited_mocks.on_welcome_message.assert_called(timeout=5)

print("ok")

23-07-04 00:19:59.882 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-07-04 00:19:59.883 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-07-04 00:19:59.885 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-04 00:19:59.886 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-04 00:19:59.887 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-04 00:19:59.888 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-07-04 00:20:00.100 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-07-04 00:20:00.101 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProduc

AssertionError: expected call not found.
Expected: mock(Employee(name='Mickey', surname='Mouse', email=None))
Actual: mock(Employee(name='Mickey', surname='Mouse', email='Mickey.Mouse@gmail.com'))